In [ ]:
import csv

import pandas as pd



Load measurements for all coins:

In [57]:
data_btc = pd.read_csv('.\\data\\btc.csv', index_col=False);
data_btc['date'] = pd.to_datetime(data_btc['date']);

data_dcr = pd.read_csv('.\\data\\dcr.csv', index_col=False);
data_dcr['date'] = pd.to_datetime(data_dcr['date']);

data_doge = pd.read_csv('.\\data\\doge.csv', index_col=False);
data_doge['date'] = pd.to_datetime(data_doge['date']);

data_etc = pd.read_csv('.\\data\\etc.csv', index_col=False);
data_etc['date'] = pd.to_datetime(data_etc['date'])

data_eth = pd.read_csv('.\\data\\eth.csv', index_col=False);
data_eth['date'] = pd.to_datetime(data_eth['date'])

data_ltc = pd.read_csv('.\\data\\ltc.csv', index_col=False);
data_ltc['date'] = pd.to_datetime(data_ltc['date'])

data_xmr = pd.read_csv('.\\data\\xmr.csv', index_col=False);
data_xmr['date'] = pd.to_datetime(data_xmr['date'])

data_xrp = pd.read_csv('.\\data\\xrp.csv', index_col=False);
data_xrp['date'] = pd.to_datetime(data_xrp['date'])

Merge data:

In [134]:
def generate_new_column_names(coin, columns):
    res = {}
    for column in columns:
        if column != 'date':
            res[column]=coin+"_"+column
    return res

def select_values_after_date(dataset, date):
    mask = (dataset['date'] > pd.to_datetime(date))
    return dataset.loc[mask]

#General columns
columns_general = list(data_btc)

#Selected date
selected_date = '2015-08-10'

#Repeat for all coins we track
#BTC
columns_btc_new = generate_new_column_names('BTC', columns_general)
new_btc = data_btc[columns_general]
new_btc = select_values_after_date(new_btc, selected_date).rename(index=str, columns=columns_btc_new);
new_btc = new_btc.reset_index(drop=True);

#DCR
columns_dcr_new = generate_new_column_names('DCR', columns_general)
new_dcr = data_dcr[columns_general]
new_dcr = select_values_after_date(new_dcr, selected_date).rename(index=str, columns=columns_dcr_new);
new_dcr = new_dcr.reset_index(drop=True);

#ETC
columns_etc_new = generate_new_column_names('ETC', columns_general)
new_etc = data_etc[columns_general]
new_etc = select_values_after_date(new_etc, selected_date).rename(index=str, columns=columns_etc_new);
new_etc = new_etc.reset_index(drop=True);

#ETH
columns_eth_new = generate_new_column_names('ETH', columns_general)
new_eth = data_eth[columns_general]
new_eth = select_values_after_date(new_eth, selected_date).rename(index=str, columns=columns_eth_new);
new_eth = new_eth.reset_index(drop=True);

#LTC
columns_ltc_new = generate_new_column_names('LTC', columns_general)
new_ltc = data_ltc[columns_general]
new_ltc = select_values_after_date(new_ltc, selected_date).rename(index=str, columns=columns_ltc_new);
new_ltc = new_ltc.reset_index(drop=True);

#XMR
columns_xmr_new = list(columns_general)
columns_xmr_new.remove('generatedCoins')
columns_xmr_new.remove('txVolume(USD)')
new_xmr = data_xmr[columns_xmr_new]
columns_xmr_new = generate_new_column_names('XMR', columns_xmr_new)
new_xmr = select_values_after_date(new_xmr, selected_date).rename(index=str, columns=columns_xmr_new);
new_xmr = new_xmr.reset_index(drop=True);

#XRP
columns_xrp_new = list(columns_general)
columns_xrp_new.remove('generatedCoins')
new_xrp = data_xrp[columns_xrp_new]
columns_xrp_new = generate_new_column_names('XRP', columns_xrp_new)
new_xrp = select_values_after_date(new_xrp, selected_date).rename(index=str, columns=columns_xrp_new);
new_xrp = new_xrp.reset_index(drop=True);

merged_data = pd.merge(new_btc, new_dcr, on='date', how='outer')
merged_data = pd.merge(merged_data, new_doge, on='date', how='outer')
merged_data = pd.merge(merged_data, new_etc, on='date', how='outer')
merged_data = pd.merge(merged_data, new_eth, on='date', how='outer')
merged_data = pd.merge(merged_data, new_ltc, on='date', how='outer')
merged_data = pd.merge(merged_data, new_xmr, on='date', how='outer')
merged_data = pd.merge(merged_data, new_xrp, on='date', how='outer')

#Save result
merged_data.to_csv('merged_data.csv');
merged_data.head()
#merged_data

,date,BTC_txVolume(USD),BTC_txCount,BTC_marketcap(USD),BTC_price(USD),BTC_exchangeVolume(USD),BTC_generatedCoins,BTC_fees,DCR_txVolume(USD),DCR_txCount,...,XMR_marketcap(USD),XMR_price(USD),XMR_exchangeVolume(USD),XMR_fees,XRP_txVolume(USD),XRP_txCount,XRP_marketcap(USD),XRP_price(USD),XRP_exchangeVolume(USD),XRP_fees
0,2015-08-11,2.799356e+08,117239,3.828500e+09,264.34,25433900.0,4200.0,26.633347,NaN,NaN,...,5823060.0,0.656747,69221.0,54.258842,4.829576e+06,21109,279220000.0,0.008751,282461.0,316.937890
1,2015-08-12,3.124407e+08,116367,3.920210e+09,270.60,26815400.0,3075.0,25.367459,NaN,NaN,...,5361380.0,0.603783,51925.0,39.381776,3.883631e+05,20774,274141000.0,0.008591,287463.0,325.955675
2,2015-08-13,3.386269e+08,117871,3.857090e+09,266.18,27685500.0,3750.0,25.047151,NaN,NaN,...,5474580.0,0.615626,51260.0,44.491584,6.157494e+05,19054,263772000.0,0.008266,202934.0,289.499482
3,2015-08-14,3.291484e+08,111030,3.828340e+09,264.13,27091200.0,3375.0,24.336399,NaN,NaN,...,5130060.0,0.575992,49540.0,20.584278,1.243518e+06,24698,259989000.0,0.008148,423163.0,371.447808
4,2015-08-15,2.545217e+08,98422,3.849530e+09,265.53,19321100.0,3350.0,20.990425,NaN,NaN,...,5453670.0,0.611454,49051.0,36.878999,1.858058e+06,21779,265790000.0,0.008330,176135.0,329.073899
